# Small Experiment Data Analysis

In [41]:
import json
from six.moves import cPickle
import os
import statistics

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [42]:
"""
Load Coco data
"""
COCO_PATH = 'data/coco/'
COCO_ANNO_PATH = COCO_PATH + 'annotations/'

# load the data from captions
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = int(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = int(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)


In [63]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_gaussian'
IN_DIR        = 'input'
OUT_DIR       = 'output'

def get_stat(data):
    if len(data)==0: return;
    print ""
    print "Total:\t%d;\t\tMean:\t\t%f;\tNonezero:\t%d"%(len(data), statistics.mean(data), len(filter(lambda x:x!=0, data)))
    print "Median:\t%f;\tMedian(H):\t%f;\tMedian(L):\t%f;"%(statistics.median(data), statistics.median_high(data), statistics.median_low(data))
    print "Max:\t%f;\tMin:\t\t%f;\tStd:\t\t%f"%(max(data), min(data),statistics.stdev(data))
    print "\n"
    
def look_up_image(title, idx, in_ids, vis, scores, out_file):
    print "[%d]%s\n"%(idx,title)
    print "\tScore:%s;\tImageId:%s\n"%(scores[idx], in_ids[idx])
    print "\tOriginal Caption:\n\t\t%s;\n\tAblation Caption:\n\t\t%s;"%(vis[idx*2+1]['caption'],vis[idx*2]['caption'])
    print "\tAnnotated Captions:\n"
    img_id = int(in_ids[idx])
    for i, note in enumerate(img_captions[img_id]['annotation']):
        print "\t\t%d. %s\n"%(i+1, str(note['caption']).strip())
    
    # Display images side by side: http://permalink.gmane.org/gmane.comp.python.ipython.devel/11073
    s = """<table>
        <tr>
        <th><img src="%s" style="max-width:400px" /></th>
        <th><img src="%s" style="max-width:400px" /></th>
        </tr></table>"""%(img_captions[img_id]['image']['coco_url'], "%s"%os.path.join("tmp_%s"%out_file, "%s_%s_ablt.jpg"%(str(idx).zfill(16),str(img_id))))
    t=HTML(s)
    display(t)

def get_expr_summary(in_file=IN_FILE_NAME, out_file=OUT_FILE_NAME, in_path=IN_DIR, out_path=OUT_DIR, num=3):
    in_ids  = cPickle.load(open(os.path.join(in_path, in_file)))
    scores  = cPickle.load(open(os.path.join(out_path, "scores_%s.pkl"%out_file)))
    vis     = json.load(open(os.path.join(out_path, 'vis_%s.json'%out_file)))
    print "Statistics for scores of gaussian filtered ablations:"
    get_stat(scores)

    scores_idx = zip(range(len(scores)), scores)
    sorted_scores_idx = sorted(scores_idx, key=lambda x: x[1])   # sort by score
    for idx, _ in sorted_scores_idx[:num]:
        look_up_image("Zero Image",idx, in_ids, vis, scores, out_file)



In [64]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_gaussian'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	1013;		Mean:		0.837452;	Nonezero:	1008
Median:	1.000000;	Median(H):	1.000000;	Median(L):	1.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.276086


[107]Zero Image

	Score:0.0;	ImageId:340096

	Original Caption:
		a stop sign is on the corner of a street;
	Ablation Caption:
		a person holding a glass of wine in a room;
	Annotated Captions:

		1. A stop sign covered in lights at a street intersection.

		2. A stop sign that is decorated with Christmas lights.

		3. a decorated lite up stop sign at the corner

		4. car comes to the curve to stop at the stop sign

		5. A stop sign covered in a white lights.



,


[427]Zero Image

	Score:0.0;	ImageId:110251

	Original Caption:
		a street sign on a street corner;
	Ablation Caption:
		a park bench sitting in the middle of a park;
	Annotated Captions:

		1. A bench sitting on top of a park near a play ground.

		2. a fenced in park on a city street

		3. a sign showing that there kids playing in the field

		4. a sign on a pole near a park

		5. A playground with a sign saying caution live children playing.



,


[550]Zero Image

	Score:0.0;	ImageId:289515

	Original Caption:
		a street sign on the side of a street;
	Ablation Caption:
		a car parked in front of a building;
	Annotated Captions:

		1. A sidewalk and doughnut shop with a bike leaning against the building

		2. A long shot of a city sidewalk on a sunny day

		3. A sign for a doughnut shop on a sidewalk by a street.

		4. Donut shop sign on an empty street next to a tree

		5. A building with a sign that says Donuts above the door.



,


In [65]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	1013;		Mean:		0.652322;	Nonezero:	976
Median:	0.714286;	Median(H):	0.714286;	Median(L):	0.714286;
Max:	1.000000;	Min:		0.000000;	Std:		0.348214


[30]Zero Image

	Score:0.0;	ImageId:178533

	Original Caption:
		an airplane is parked on a runway in a field;
	Ablation Caption:
		a group of people standing around a plane;
	Annotated Captions:

		1. A group of pilots posing in front of a fighter jet.

		2. A fighter jet parked behind U.S. Air Force pilots.

		3. Air Force member posed in front of a jet

		4. a group of people kneeling down next to an air force jet plane

		5. A group of pilots posing in front of a jet.



,


[62]Zero Image

	Score:0.0;	ImageId:8379

	Original Caption:
		a street sign on the side of the road;
	Ablation Caption:
		a tv sitting in the middle of a forest;
	Annotated Captions:

		1. An old "Parking" sign is attached to a rusted pole.

		2. A PICTURE OF A PARKING SIGN ON THE STREET

		3. An old rusty pole that has a sign "Parking" hanging from it. Across from it is an old brick building with three windows.

		4. a pole with a sign underneath a tree and in front of a building

		5. a parking sighn on a feild next to a house.



,


[63]Zero Image

	Score:0.0;	ImageId:466941

	Original Caption:
		a sign that says UNK UNK on the side of it;
	Ablation Caption:
		a fire hydrant in the middle of a forest;
	Annotated Captions:

		1. A tall sign advertising Cider in front of a net.

		2. A large sign is advertising cold fresh cider.

		3. An apple cider sign between colorful Autumn foliage.

		4. A painted sign for cold, fresh cider sits outside.

		5. This is the place to stop for cold cider on a hot day.



,


In [66]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_gaussian'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	683;		Mean:		0.818877;	Nonezero:	681
Median:	1.000000;	Median(H):	1.000000;	Median(L):	1.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.279659


[378]Zero Image

	Score:0.0;	ImageId:247708

	Original Caption:
		a person holding a cell phone in their hand;
	Ablation Caption:
		a laptop computer sitting on top of a table;
	Annotated Captions:

		1. a person operating a cell phone near a laptop

		2. A laptop, smartphone, and iPad all on a table

		3. A person is calling someone on his phone regarding information on his laptop.

		4. A person working on a computer who is holding an iPhone.

		5. A person multitasking by using a laptop, phone, and tablet.



,


[480]Zero Image

	Score:0.0;	ImageId:546211

	Original Caption:
		a man is skiing down a snowy hill;
	Ablation Caption:
		a group of people standing on top of a snow covered slope;
	Annotated Captions:

		1. A skier is standing beside a rack of skis and snowboards.

		2. A skier looks at a wooden rack of skis at the bottom of a slope.

		3. A person on some skis in the snow.

		4. Skiers coming down a slope on a sunny day.

		5. A man skiing next to a ski rack.



,


[150]Zero Image

	Score:0.0833333333333;	ImageId:132883

	Original Caption:
		a street sign on a pole on a city street;
	Ablation Caption:
		a red and white building with a clock on the side of it;
	Annotated Captions:

		1. A red and white street sign sitting in front of a church.

		2. A sign with spray paint all over it .

		3. The sign describes the hours of the attraction.

		4. A red and white sign is all scribbled on.

		5. Vandalism on a sign with words that are not in English.



,


In [67]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	683;		Mean:		0.521510;	Nonezero:	630
Median:	0.500000;	Median(H):	0.500000;	Median(L):	0.500000;
Max:	1.000000;	Min:		0.000000;	Std:		0.352480


[23]Zero Image

	Score:0.0;	ImageId:330436

	Original Caption:
		a man is doing a trick on a skateboard;
	Ablation Caption:
		a person jumping a skate board in the air;
	Annotated Captions:

		1. a person jumping a skate board in the air

		2. This skateboarder is riding the side of his skateboard in the air.

		3. The young man is riding his skateboard practicing his moves.

		4. A skateboarder is airborne in front of a wall covered in graffiti.

		5. A person jumping in the air on a skateboard.



,


[27]Zero Image

	Score:0.0;	ImageId:375294

	Original Caption:
		a couple of cell phones sitting on top of a table;
	Ablation Caption:
		a person is holding a small piece of paper;
	Annotated Captions:

		1. there are 2 tablets, 1 e-reader, and an iphone sitting on the table.

		2. a table that has a bunch of ipads on it

		3. some different tablets cellphone and a piece of pizza

		4. Tablets and a cell phone on a table beside a plate of pizza.

		5. Some iPads and phone are on a table with pizza.



,


[33]Zero Image

	Score:0.0;	ImageId:172316

	Original Caption:
		a young boy swinging a baseball bat at a ball;
	Ablation Caption:
		a man standing in front of a wall with a cell phone;
	Annotated Captions:

		1. A boy is swinging at a ball during a baseball game.

		2. A boy that is hitting a ball with a baseball bat.

		3. A person with a bat and helmet hits a ball.

		4. a boy in a yellow and white uniform hitting a baseball

		5. A young baseball player is hitting the ball.



,
